In [41]:
import numpy as np
import pandas as pd
from quantstats import extend_pandas
extend_pandas()

In [42]:
df = pd.read_csv("JALSH Index_dataset_2000_2024_clean.csv", index_col=0, header=[0, 1])
classfier = pd.read_excel("classification_data.xlsx", index_col=0)

In [3]:
print("Sector")
print(classfier.sector.unique())
print("Industry")
print(classfier.industry.unique())

Sector
['Financial' 'Consumer, Non-cyclical' 'Industrial' 'Basic Materials'
 'Diversified' 'Consumer, Cyclical' 'Communications' 'Technology' 'Energy']
Industry
['Banks' 'Commercial Services' 'Electronics' 'Miscellaneous Manufactur'
 'Diversified Finan Serv' 'Building Materials' 'Mining'
 'Investment Companies' 'Pharmaceuticals' 'Beverages' 'Agriculture'
 'REITS' 'Holding Companies-Divers' 'Distribution/Wholesale' 'Food'
 'Telecommunications' 'Computers' 'Retail' 'Insurance' 'Real Estate'
 'Coal' 'Transportation' 'Entertainment' 'Auto Parts&Equipment'
 'Iron/Steel' 'Software' 'Healthcare-Services' 'Private Equity'
 'Energy-Alternate Sources' 'Forest Products&Paper' 'Internet' 'Chemicals'
 'Engineering&Construction' 'Lodging']


In [4]:
profitability = ['ebitda_margin', 'oper_margin', 'return_on_asset']
liquidity = ['tot_debt_to_ebitda', 'tot_debt_to_tot_asset', 'tot_debt_to_tot_cap', 'tot_debt_to_tot_eqy', 'interest_coverage_ratio', 'ebitda_to_tot_int_exp']
efficiency = ['invent_to_sales', 'asset_turnover']
bloomberg_metrics = ['tot_debt_to_tot_eqy', 'interest_coverage_ratio', 'return_on_asset', 'tot_debt_to_ebitda', 'ebitda_to_tot_int_exp']

In [5]:
def filter_securities(filters, data=classfier):
    
    # Apply filters directly on the transposed
    filtered_data = data.query(
        ' and '.join(f'`{k}` == "{v}"' for k, v in filters.items())
    )
    
    return list(filtered_data.index)


def get_metrics(df):
    df = df.copy()
    individual_metrics = {}

    # List all metrics together for iteration
    all_metrics = profitability + liquidity + efficiency

    # Calculate the average of each metric individually for each stock
    for stock in df.columns.levels[0]:
        individual_metrics[stock] = {}
        for metric in all_metrics:
            if metric in df[stock].columns:
                individual_metrics[stock][metric] = df[stock][metric].mean()

    # Convert the results to a DataFrame for better presentation
    return pd.DataFrame(individual_metrics)

In [10]:
securities = filter_securities({"sector":"Communications", "industry": "Telecommunications"})

metrics = get_metrics(df)
sector_secuirties = metrics.loc[bloomberg_metrics, securities]

In [11]:
sector_secuirties

,BLU SJ Equity,MTN SJ Equity,TKG SJ Equity,VOD SJ Equity
tot_debt_to_tot_eqy,32.809328,65.116785,56.593172,65.004940
interest_coverage_ratio,6.346575,8.547827,3.495262,9.790341
return_on_asset,4.215240,7.405181,6.401370,16.143841
tot_debt_to_ebitda,1.150158,1.305329,1.430977,0.793112
ebitda_to_tot_int_exp,7.547418,13.114130,8.796523,13.355185


In [12]:
expert_thresholds = {
  "tot_debt_to_tot_eqy": {
    1: (0, 0.5),
    0: (0.5, 2),
    -1: (2, float("inf")),
  },
  "interest_coverage_ratio": {
    1: (8, float("inf")),
    0: (3, 8),
    -1: (0, 3),
  },
  "return_on_asset": {
    1: (0.2, float("inf")),
    0: (0.05, 0.2),
    -1: (0, 0.05),
  },
  "tot_debt_to_ebitda": {
    1: (0, 2),
    0: (2, 4),
    -1: (4, float("inf")),
  },
  "ebitda_to_tot_int_exp": {
    1: (8, float("inf")),
    0: (3, 8),
    -1: (0, 3),
  },
}


In [13]:
def create_industry_thresholds(df, metric_directions):
    industry_thresholds = {}

    for metric in df.index:
        q25 = df.loc[metric].quantile(0.25)
        q75 = df.loc[metric].quantile(0.75)
        
        mn = df.loc[metric].min()
        mx = df.loc[metric].max()
        
        if metric_directions[metric] == "higher_better":
            industry_thresholds[metric] = {
                1: (q75, np.inf),
                0: (q25, q75),
                -1: (-np.inf, q25)
            }
        elif metric_directions[metric] == "lower_better":
            industry_thresholds[metric] = {
                1: (-np.inf, q25),
                0: (q25, q75),
                -1: (q75, np.inf)
            }
        else:
            raise ValueError(f"Invalid direction specified for metric '{metric}'")

    return industry_thresholds

def classify_stock_metrics(df, thresholds):
    classified_df = df.copy()

    for metric, threshold in thresholds.items():
        if metric in df.index:
            for category, bounds in threshold.items():
                lower, upper = bounds
                classified_df.loc[metric] = classified_df.loc[metric].apply(
                    lambda x: category if lower <= float(x) < upper else classified_df.loc[metric, classified_df.columns[0]]
                )
        else:
            print(f"Metric '{metric}' not found in the DataFrame. Skipping classification for this metric.")

    return classified_df

def assess_creditworthiness(classified_metrics, weights):
    scores = {
        1: 3,
        0: 2,
        -1: 1
    }

    creditworthiness_scores = {}
    default_probabilities = {}

    for stock in classified_metrics.columns:
        stock_score = 0
        for metric, weight in weights.items():
            if metric in classified_metrics.index:
                category = classified_metrics.loc[metric, stock]
                stock_score += scores[category] * weight
            else:
                print(f"Metric '{metric}' not found for stock '{stock}'. Skipping this metric.")
        
        creditworthiness_scores[stock] = stock_score

        # Calculate the probability of default
        max_score = sum(weight * 3 for weight in weights.values())
        min_score = sum(weight * 1 for weight in weights.values())
        normalized_score = (stock_score - min_score) / (max_score - min_score)
        default_probability = 1 - normalized_score
        default_probabilities[stock] = default_probability

    return creditworthiness_scores

In [29]:
metric_directions = {
    "tot_debt_to_tot_eqy": "lower_better",
    "interest_coverage_ratio": "higher_better",
    "return_on_asset": "higher_better",
    "tot_debt_to_ebitda": "lower_better",
    "ebitda_to_tot_int_exp": "higher_better"
}

weights = {
    "tot_debt_to_tot_eqy": 0.25,
    "interest_coverage_ratio": 0.25,
    "return_on_asset": 0.2,
    "tot_debt_to_ebitda": 0.15,
    "ebitda_to_tot_int_exp": 0.15
}

# weights = {
#     "tot_debt_to_tot_eqy": 0.25,  
#     "interest_coverage_ratio": 0.30,  
#     "return_on_asset": 0.10,  
#     "tot_debt_to_ebitda": 0.25,  
#     "ebitda_to_tot_int_exp": 0.10 
# }


industry_thresholds = create_industry_thresholds(sector_secuirties, metric_directions)
classified_metrics = classify_stock_metrics(sector_secuirties, industry_thresholds) # or expert_thresholds
creditworthiness_scores = assess_creditworthiness(classified_metrics, weights)

In [30]:
print("Creditworthiness Scores:")
print(creditworthiness_scores)

Creditworthiness Scores:
{'BLU SJ Equity': 1.65, 'MTN SJ Equity': 1.65, 'TKG SJ Equity': 1.65, 'VOD SJ Equity': 1.9}


In [43]:
def create_industry_thresholds_ratios(df, metric_directions):
    industry_thresholds = {}

    for metric in df.index:
        q25 = df.loc[metric].quantile(0.25)
        q75 = df.loc[metric].quantile(0.75)
        
        mn = df.loc[metric].min()
        mx = df.loc[metric].max()
        
        if metric_directions[metric] == "higher_better":
            industry_thresholds[metric] = q25, np.inf,
                
        elif metric_directions[metric] == "lower_better":
            industry_thresholds[metric] = -np.inf, q25
        else:
            raise ValueError(f"Invalid direction specified for metric '{metric}'")
    return industry_thresholds

In [45]:
rh = create_industry_thresholds_ratios(sector_secuirties, metric_directions)

In [46]:
def score_ratio(ratio, healthy_range, inverse=False):
    """
    Converts a financial ratio to a score based on its deviation from a healthy range.
    :param ratio: The financial ratio value.
    :param healthy_range: A tuple indicating the healthy range of values for the ratio.
    :param inverse: Set to True if lower values are better for the ratio.
    :return: A score as a float.
    """
    if inverse:
        if ratio < healthy_range[0]:
            return (healthy_range[0] - ratio) / healthy_range[0]
        elif ratio > healthy_range[1]:
            return -(ratio - healthy_range[1]) / healthy_range[1]
    else:
        if ratio > healthy_range[1]:
            return (ratio - healthy_range[1]) / healthy_range[1]
        elif ratio < healthy_range[0]:
            return -(healthy_range[0] - ratio) / healthy_range[0]
    return 0  # Ratio is within the healthy range

In [60]:
df = sector_secuirties.T.copy()

In [61]:
industry_thresholds

{'tot_debt_to_tot_eqy': {1: (-inf, 50.647210600805735),
  0: (50.647210600805735, 65.03290102599539),
  -1: (65.03290102599539, inf)},
 'interest_coverage_ratio': {1: (8.858455386626364, inf),
  0: (5.633747012489427, 8.858455386626364),
  -1: (-inf, 5.633747012489427)},
 'return_on_asset': {1: (9.589846015144378, inf),
  0: (5.854837291308153, 9.589846015144378),
  -1: (-inf, 5.854837291308153)},
 'tot_debt_to_ebitda': {1: (-inf, 1.060896398933337),
  0: (1.060896398933337, 1.3367408348954506),
  -1: (1.3367408348954506, inf)},
 'ebitda_to_tot_int_exp': {1: (13.174393594464583, inf),
  0: (8.484247144183694, 13.174393594464583),
  -1: (-inf, 8.484247144183694)}}

In [62]:
# Classification values and ranges
classification_values = {'Poor': 1, 'Fair': 2, 'Good': 3}

classification_ranges = {
    'Return_on_Assets': {'Poor': (None, 0.02), 'Fair': (0.02, 0.05), 'Good': (0.05, None)},
    'Operating_Margin_Ratio': {'Poor': (None, 0.10), 'Fair': (0.10, 0.20), 'Good': (0.20, None)},
    'Net_Profit_Margin': {'Poor': (None, 0.05), 'Fair': (0.05, 0.10), 'Good': (0.10, None)},
    'Debt_to_EBITDA': {'Good': (None, 2.0), 'Fair': (2.0, 4.0), 'Poor': (4.0, None)},  # Note: Inverted logic for Debt to EBITDA
    'EBITDA_to_Interest_Expense': {'Poor': (None, 2.0), 'Fair': (2.0, 4.0), 'Good': (4.0, None)},
    'Asset_Turnover': {'Poor': (None, 0.5), 'Fair': (0.5, 1.5), 'Good': (1.5, None)}
}

# Function to classify metric values into "Poor", "Fair", and "Good"
def classify_metric(value, metric):
    ranges = classification_ranges[metric]
    for classification, (low, high) in ranges.items():
        if low is None and value <= high:
            return classification
        elif high is None and value > low:
            return classification
        elif low is not None and high is not None and low <= value <= high:
            return classification
    return "Unknown"

# Apply classification for each metric
for metric in industry_thresholds.keys():
    df[f'{metric}_Class'] = df[metric].apply(lambda x: classify_metric(x, metric))

# # Function to calculate overall score and classification
# def calculate_and_classify_overall(row):
#     scores = [classification_values[row[f'{metric}_Class']] for metric in classification_ranges]
#     overall_score = sum(scores) / len(scores)
#     overall_class = 'Good' if overall_score > 2.5 else 'Fair' if overall_score > 1.5 else 'Poor'
#     return pd.Series([overall_score, overall_class], index=['Overall_Score', 'Overall_Class'])

# # Calculate overall scores and classifications
# df[['Overall_Score', 'Overall_Class']] = df.apply(calculate_and_classify_overall, axis=1)

KeyError: 'tot_debt_to_tot_eqy'

In [64]:
df["tot_debt_to_tot_eqy"]

BLU SJ Equity    32.809328
MTN SJ Equity    65.116785
TKG SJ Equity    56.593172
VOD SJ Equity    65.004940
Name: tot_debt_to_tot_eqy, dtype: float64

In [18]:
def calculate_credit_score(financial_ratios, industry_thresholds, expert_thresholds, weights):
    # Calculate the weighted average
    weighted_average = sum(ratio * weight for ratio, weight in zip(financial_ratios, weights))
    
    # Calculate the normalized distances to each threshold
    distances = {
        rating: abs(weighted_average - (industry_thresholds[rating] + expert_thresholds[rating]) / 2) / (max(industry_thresholds[rating], expert_thresholds[rating]) - min(industry_thresholds[rating], expert_thresholds[rating]))
        for rating in ['good', 'fair', 'poor']
    }
    print(distances)
    # Find the rating with the minimum distance
    credit_rating = min(distances, key=distances.get)
    
    # Assign the credit score based on the rating
    credit_score = {'good': 3, 'fair': 2, 'poor': 1}[credit_rating]
    
    # Calculate the confidence level based on the distance to the assigned rating
    confidence_level = 1 - distances[credit_rating]
    
    return credit_score, credit_rating, confidence_level

def get_credit_recommendation(credit_score, credit_rating, confidence_level, threshold=0.8):
    if confidence_level >= threshold:
        recommendation = f"The entity has a {credit_rating} credit rating with a high confidence level."
    else:
        recommendation = f"The entity has a {credit_rating} credit rating, but the confidence level is low. Further investigation is recommended."
    
    return recommendation

# Example usage
financial_ratios = [0.8, 1.5, 2.0, 0.6]
industry_thresholds = {'good': 1.0, 'fair': 0.7, 'poor': 0.4}
expert_thresholds = {'good': 1.2, 'fair': 0.8, 'poor': 0.5}
weights = [0.3, 0.2, 0.3, 0.2]

credit_score, credit_rating, confidence_level = calculate_credit_score(financial_ratios, industry_thresholds, expert_thresholds, weights)
recommendation = get_credit_recommendation(credit_score, credit_rating, confidence_level)

print("Credit Score:", credit_score)
print("Credit Rating:", credit_rating)
print("Confidence Level:", confidence_level)
print("Recommendation:", recommendation)

{'good': 0.8000000000000009, 'fair': 5.099999999999998, 'poor': 8.100000000000005}
Credit Score: 3
Credit Rating: good
Confidence Level: 0.19999999999999907
Recommendation: The entity has a good credit rating, but the confidence level is low. Further investigation is recommended.


In [7]:
from scipy.spatial import distance

def calculate_credit_score(financial_ratios, industry_thresholds, expert_thresholds, weights):
    # Calculate the weighted average
    weighted_average = sum(ratio * weight for ratio, weight in zip(financial_ratios, weights))
    
    # Calculate the Euclidean distances to each threshold
    distances = {
        rating: distance.euclidean([weighted_average], [(industry_thresholds[rating] + expert_thresholds[rating]) / 2])
        for rating in ['good', 'fair', 'poor']
    }
    
    # Find the rating with the minimum distance
    credit_rating = min(distances, key=distances.get)
    
    # Calculate the credit score based on the weighted average
    max_score = 100
    min_score = 0
    credit_score = (weighted_average - min(industry_thresholds.values())) / (max(industry_thresholds.values()) - min(industry_thresholds.values())) * (max_score - min_score) + min_score
    credit_score = round(credit_score)
    
    return credit_score, credit_rating

def get_credit_recommendation(credit_score, credit_rating):
    if credit_score >= 80:
        recommendation = f"The entity has a {credit_rating} credit rating and a high credit score of {credit_score}. It is considered a low-risk entity."
    elif credit_score >= 60:
        recommendation = f"The entity has a {credit_rating} credit rating and a moderate credit score of {credit_score}. It is considered a medium-risk entity."
    else:
        recommendation = f"The entity has a {credit_rating} credit rating and a low credit score of {credit_score}. It is considered a high-risk entity."
    
    return recommendation

# Example usage
financial_ratios = [0.8, 1.5, 2.0, 0.6]
industry_thresholds = {'good': 1.0, 'fair': 0.7, 'poor': 0.4}
expert_thresholds = {'good': 1.2, 'fair': 0.8, 'poor': 0.5}
weights = [0.3, 0.2, 0.3, 0.2]

credit_score, credit_rating = calculate_credit_score(financial_ratios, industry_thresholds, expert_thresholds, weights)
recommendation = get_credit_recommendation(credit_score, credit_rating)

print("Credit Score:", credit_score)
print("Credit Rating:", credit_rating)
print("Recommendation:", recommendation)

Credit Score: 143
Credit Rating: good
Recommendation: The entity has a good credit rating and a high credit score of 143. It is considered a low-risk entity.


In [9]:
from scipy.spatial import distance

def calculate_credit_score(financial_ratios, industry_thresholds, expert_thresholds, weights):
    """
    Calculate the credit score and rating based on financial ratios and thresholds.

    Args:
        financial_ratios (list): List of financial ratios.
        industry_thresholds (dict): Dictionary of industry thresholds for each rating.
        expert_thresholds (dict): Dictionary of expert thresholds for each rating.
        weights (list): List of weights corresponding to each financial ratio.

    Returns:
        tuple: A tuple containing the credit score (int) and credit rating (str).
    """
    weighted_average = sum(ratio * weight for ratio, weight in zip(financial_ratios, weights))

    distances = {
        rating: distance.euclidean(
            [weighted_average],
            [(industry_thresholds[rating] + expert_thresholds[rating]) / 2]
        )
        for rating in ['good', 'fair', 'poor']
    }

    credit_rating = min(distances, key=distances.get)

    max_score = 100
    min_score = 0
    min_threshold = min(industry_thresholds.values())
    max_threshold = max(industry_thresholds.values())

    credit_score = (weighted_average - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score
    credit_score = round(credit_score)

    return credit_score, credit_rating


def get_credit_recommendation(credit_score, credit_rating):
    """
    Generate a credit recommendation based on the credit score and rating.

    Args:
        credit_score (int): Credit score value.
        credit_rating (str): Credit rating category.

    Returns:
        str: Credit recommendation message.
    """
    if credit_score >= 80:
        risk_level = "low"
    elif credit_score >= 60:
        risk_level = "medium"
    else:
        risk_level = "high"

    recommendation = (
        f"The entity has a {credit_rating} credit rating and a {risk_level} credit score of {credit_score}. "
        f"It is considered a {risk_level}-risk entity."
    )

    return recommendation


# Example usage
if __name__ == "__main__":
    financial_ratios = [0.8, 1.5, 2.0, 0.6]
    industry_thresholds = {'good': 1.0, 'fair': 0.7, 'poor': 0.4}
    expert_thresholds = {'good': 1.2, 'fair': 0.8, 'poor': 0.5}
    weights = [0.3, 0.2, 0.3, 0.2]

    credit_score, credit_rating = calculate_credit_score(financial_ratios, industry_thresholds, expert_thresholds, weights)
    recommendation = get_credit_recommendation(credit_score, credit_rating)

    print(f"Credit Score: {credit_score}")
    print(f"Credit Rating: {credit_rating}")
    print(f"Recommendation: {recommendation}")

Credit Score: 143
Credit Rating: good
Recommendation: The entity has a good credit rating and a low credit score of 143. It is considered a low-risk entity.


In [13]:
distance.euclidean([10, 11], [9, 10])

1.4142135623730951

In [15]:
distance.braycurtis([10, 11], [9, 10])

0.05

In [16]:
10-9

1

In [27]:
from scipy.spatial import distance

def calculate_credit_score(financial_ratios, industry_thresholds, expert_thresholds, weights):
    """
    Calculate the credit score, rating, and confidence level based on financial ratios and thresholds.

    Args:
        financial_ratios (list): List of financial ratios.
        industry_thresholds (dict): Dictionary of industry thresholds for each rating.
        expert_thresholds (dict): Dictionary of expert thresholds for each rating.
        weights (list): List of weights corresponding to each financial ratio.

    Returns:
        tuple: A tuple containing the credit score (int), credit rating (str),
               confidence level (float), and rating scales (dict).
    """
    weighted_average = sum(ratio * weight for ratio, weight in zip(financial_ratios, weights))

    distances = {
        rating: distance.euclidean(
            [weighted_average],
            [(industry_thresholds[rating] + expert_thresholds[rating]) / 2]
        )
        for rating in ['good', 'fair', 'poor']
    }

    credit_rating = min(distances, key=distances.get)

    max_score = 100
    min_score = 0
    min_threshold = min(industry_thresholds.values())
    max_threshold = max(industry_thresholds.values())

    credit_score = (weighted_average - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score
    credit_score = round(credit_score)

    confidence_level = 1 - distances[credit_rating] / (max_threshold - min_threshold)

    return credit_score, credit_rating, confidence_level


def get_credit_recommendation(credit_score, credit_rating, confidence_level):
    """
    Generate a credit recommendation based on the credit score, rating, and confidence level.

    Args:
        credit_score (int): Credit score value.
        credit_rating (str): Credit rating category.
        confidence_level (float): Confidence level of the credit rating.

    Returns:
        str: Credit recommendation message.
    """
    if credit_score >= 80:
        risk_level = "low"
    elif credit_score >= 60:
        risk_level = "medium"
    else:
        risk_level = "high"

    recommendation = (
        f"The entity has a {credit_rating} credit rating with a confidence level of {confidence_level:.2%}. "
        f"The credit score of {credit_score} indicates a {risk_level} risk level."
    )

    return recommendation


# Example usage
if __name__ == "__main__":
    financial_ratios = [0.8, 1.5, 2.0, 4.6]
    industry_thresholds = {'good': 1.0, 'fair': 0.7, 'poor': 0.4}
    expert_thresholds = {'good': 1.2, 'fair': 0.8, 'poor': 0.5}
    weights = [0.3, 0.2, 0.3, 0.2]

    credit_score, credit_rating, confidence_level, rating_scales = calculate_credit_score(
        financial_ratios, industry_thresholds, expert_thresholds, weights
    )
    recommendation = get_credit_recommendation(credit_score, credit_rating, confidence_level)

    print(f"Credit Score: {credit_score}")
    print(f"Credit Rating: {credit_rating}")
    print(f"Confidence Level: {confidence_level:.2%}")
    print(f"Rating Scales: {rating_scales}")
    print(f"Recommendation: {recommendation}")

ValueError: not enough values to unpack (expected 4, got 3)

In [31]:
from scipy.spatial import distance

def calculate_credit_score(financial_ratios, industry_thresholds, expert_thresholds, weights):
    """
    Calculate the credit score, rating, confidence level, and range scores based on financial ratios and thresholds.

    Args:
        financial_ratios (list): List of financial ratios.
        industry_thresholds (dict): Dictionary of industry thresholds for each rating.
        expert_thresholds (dict): Dictionary of expert thresholds for each rating.
        weights (list): List of weights corresponding to each financial ratio.

    Returns:
        tuple: A tuple containing the credit score (int), credit rating (str),
               confidence level (float), rating scales (dict), and range scores (dict).
    """
    weighted_average = sum(ratio * weight for ratio, weight in zip(financial_ratios, weights))

    distances = {
        rating: distance.euclidean(
            [weighted_average],
            [(industry_thresholds[rating] + expert_thresholds[rating]) / 2]
        )
        for rating in ['good', 'fair', 'poor']
    }

    credit_rating = min(distances, key=distances.get)

    max_score = 100
    min_score = 0
    min_threshold = min(industry_thresholds.values())
    max_threshold = max(industry_thresholds.values())

    credit_score = (weighted_average - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score
    credit_score = round(credit_score)

    confidence_level = 1 - distances[credit_rating] / (max_threshold - min_threshold)

    range_scores = {
        'good': {
            'min': (industry_thresholds['good'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score,
            #'max': max_score
        },
        'fair': {
            'min': (industry_thresholds['fair'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score,
            'max': (industry_thresholds['good'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score - 1
        },
        'poor': {
            'min': min_score,
            'max': (industry_thresholds['fair'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score - 1
        }
    }

    return credit_score, credit_rating, confidence_level, range_scores

In [32]:
credit_score, credit_rating, confidence_level, range_scores = calculate_credit_score(
    financial_ratios, industry_thresholds, expert_thresholds, weights
)

In [33]:
range_scores

{'good': {'min': 100.0},
 'fair': {'min': 49.999999999999986, 'max': 99.0},
 'poor': {'min': 0, 'max': 48.999999999999986}}

In [34]:
from scipy.spatial import distance

def calculate_credit_score(financial_ratio, industry_thresholds, expert_thresholds):
    """
    Calculate the credit score, rating, confidence level, and range scores based on a single financial ratio and thresholds.

    Args:
        financial_ratio (float): Financial ratio value.
        industry_thresholds (dict): Dictionary of industry thresholds for each rating.
        expert_thresholds (dict): Dictionary of expert thresholds for each rating.

    Returns:
        tuple: A tuple containing the credit score (int), credit rating (str),
               confidence level (float), rating scales (dict), and range scores (dict).
    """
    distances = {
        rating: distance.euclidean(
            [financial_ratio],
            [(industry_thresholds[rating] + expert_thresholds[rating]) / 2]
        )
        for rating in ['good', 'fair', 'poor']
    }

    credit_rating = min(distances, key=distances.get)

    max_score = 100
    min_score = 0
    min_threshold = min(industry_thresholds.values())
    max_threshold = max(industry_thresholds.values())

    credit_score = (financial_ratio - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score
    credit_score = round(credit_score)

    confidence_level = 1 - distances[credit_rating] / (max_threshold - min_threshold)

    rating_scales = {
        'good': {'min': industry_thresholds['good'], 'max': max_threshold},
        'fair': {'min': industry_thresholds['fair'], 'max': industry_thresholds['good']},
        'poor': {'min': min_threshold, 'max': industry_thresholds['fair']}
    }

    range_scores = {
        'good': {
            'min': (industry_thresholds['good'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score,
            'max': max_score
        },
        'fair': {
            'min': (industry_thresholds['fair'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score,
            'max': (industry_thresholds['good'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score - 1
        },
        'poor': {
            'min': min_score,
            'max': (industry_thresholds['fair'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score - 1
        }
    }

    return credit_score, credit_rating, confidence_level, rating_scales, range_scores

In [45]:
financial_ratio = 0.61
industry_thresholds = {'good': 1.0, 'fair': 0.7, 'poor': 0.4}
expert_thresholds = {'good': 1.2, 'fair': 0.8, 'poor': 0.5}

In [46]:
calculate_credit_score(financial_ratio, industry_thresholds, expert_thresholds)

(35,
 'fair',
 0.7666666666666666,
 {'good': {'min': 1.0, 'max': 1.0},
  'fair': {'min': 0.7, 'max': 1.0},
  'poor': {'min': 0.4, 'max': 0.7}},
 {'good': {'min': 100.0, 'max': 100},
  'fair': {'min': 49.999999999999986, 'max': 99.0},
  'poor': {'min': 0, 'max': 48.999999999999986}})

\begin{align*}
\bar{r} &= \sum_{i=1}^{n} r_i w_i \\
d_j &= \sqrt{(\bar{r} - \frac{t_{ij} + t_{ej}}{2})^2} \\
j^* &= \min_{j \in \{g, f, p\}} d_j \\
t_{\min} &= \min_{j \in \{g, f, p\}} t_{ij} \\
t_{\max} &= \max_{j \in \{g, f, p\}} t_{ij} \\
s &= \frac{\bar{r} - t_{\min}}{t_{\max} - t_{\min}} (s_{\max} - s_{\min}) + s_{\min} \\
s &= \lfloor s \rceil
\end{align*}


Notations:

$r_i$: financial ratio $i$

$w_i$: weight corresponding to financial ratio $i$

$\bar{r}$: weighted average of financial ratios

$d_j$: Euclidean distance for rating $j$

$t_{ij}$: industry threshold for rating $j$

$t_{ej}$: expert threshold for rating $j$

$j^*$: credit rating with the minimum distance

$t_{\min}$: minimum threshold value

$t_{\max}$: maximum threshold value

$s$: credit score

$s_{\min}$: minimum credit score (0)

$s_{\max}$: maximum credit score (100)

$\lfloor \cdot \rceil$: rounding function

The function returns a tuple $(s, j^)$ containing the credit score $s$ and the credit rating $j^$.

\begin{align*}
\bar{r} &= \sum_{i=1}^{n} r_i w_i \\
d_j &= \sqrt{(\bar{r} - \frac{t_{ij} + t_{ej}}{2})^2} \\
j^* &= \min_{j \in \{g, f, p\}} d_j \\
t_{\min} &= \min_{j \in \{g, f, p\}} t_{ij} \\
t_{\max} &= \max_{j \in \{g, f, p\}} t_{ij} \\
s &= \frac{\bar{r} - t_{\min}}{t_{\max} - t_{\min}} (s_{\max} - s_{\min}) + s_{\min} \\
s &= \lfloor s \rceil
\end{align*}



Notations:

$r_i$: financial ratio $i$
$w_i$: weight corresponding to financial ratio $i$
$\bar{r}$: weighted average of financial ratios
$d_j$: Euclidean distance for rating $j$
$t_{ij}$: industry threshold for rating $j$
$t_{ej}$: expert threshold for rating $j$
$j^*$: credit rating with the minimum distance
$t_{\min}$: minimum threshold value
$t_{\max}$: maximum threshold value
$s$: credit score
$s_{\min}$: minimum credit score (0)
$s_{\max}$: maximum credit score (100)
$\lfloor \cdot \rceil$: rounding function
The function returns a tuple $(s, j^)$ containing the credit score $s$ and the credit rating $j^$.

In [98]:
from scipy.spatial import distance

def calculate_credit_score(financial_ratio, industry_thresholds, expert_thresholds, inverse_relationship=False):
    """
    Calculate the credit score, rating, confidence level, and range scores based on a single financial ratio and thresholds.

    Args:
        financial_ratio (float): Financial ratio value.
        industry_thresholds (dict): Dictionary of industry thresholds for each rating.
        expert_thresholds (dict): Dictionary of expert thresholds for each rating.
        inverse_relationship (bool): Indicates if the financial ratio has an inverse relationship (lower is better).

    Returns:
        tuple: A tuple containing the credit score (int), credit rating (str),
               confidence level (float), rating scales (dict), and range scores (dict).
    """
    if inverse_relationship:
        financial_ratio = 1 / financial_ratio

    distances = {
        rating: distance.euclidean(
            [financial_ratio],
            [(industry_thresholds[rating] + expert_thresholds[rating]) / 2]
        )
        for rating in ['good', 'fair', 'poor']
    }

    credit_rating = min(distances, key=distances.get)
    
    print

    max_score = 100
    min_score = 0
    min_threshold = min(industry_thresholds.values())
    max_threshold = max(industry_thresholds.values())

    credit_score = (financial_ratio - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score
    credit_score = round(abs(credit_score))
    
    if credit_score < max_score:
        confidence_level = 1 - distances[credit_rating] / (max_threshold - min_threshold)
    else:
        confidence_level = 1.0

    range_scores = {
        'good': {
            'min': (industry_thresholds['good'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score,
            'max': max_score
        },
        'fair': {
            'min': (industry_thresholds['fair'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score,
            'max': (industry_thresholds['good'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score - 1
        },
        'poor': {
            'min': min_score,
            'max': (industry_thresholds['fair'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score - 1
        }
    }

    return credit_score, credit_rating, confidence_level, rating_scales, range_scores


def get_credit_recommendation(credit_score, credit_rating, confidence_level):
    """
    Generate a credit recommendation based on the credit score, rating, and confidence level.

    Args:
        credit_score (int): Credit score value.
        credit_rating (str): Credit rating category.
        confidence_level (float): Confidence level of the credit rating.

    Returns:
        str: Credit recommendation message.
    """
    if credit_score >= 80:
        risk_level = "low"
    elif credit_score >= 60:
        risk_level = "medium"
    else:
        risk_level = "high"

    recommendation = (
        f"The entity has a {credit_rating} credit rating with a confidence level of {confidence_level:.2%}. "
        f"The credit score of {credit_score} indicates a {risk_level} risk level."
    )

    return recommendation

In [91]:
# ... (previous code remains the same)

# Example usage
if __name__ == "__main__":
    # Define financial ratios for each company
    company_ratios = {
        'A': [1.2, 0.8, 1.5, 0.6, 1.0],
        'B': [0.9, 1.1, 1.3, 0.7, 0.8],
        'C': [1.0, 0.6, 0.1, 0.9, 0.2]
    }

    # Define industry and expert thresholds for each financial ratio
    industry_thresholds = [
        {'good': 1.0, 'fair': 0.7, 'poor': 0.4},
        {'good': 0.6, 'fair': 0.4, 'poor': 0.2},
        {'good': 1.2, 'fair': 0.9, 'poor': 0.6},
        {'good': 0.8, 'fair': 0.6, 'poor': 0.4},
        {'good': 1.1, 'fair': 0.8, 'poor': 0.5}
    ]

    expert_thresholds = [
        {'good': 1.2, 'fair': 0.8, 'poor': 0.5},
        {'good': 0.7, 'fair': 0.5, 'poor': 0.3},
        {'good': 1.4, 'fair': 1.0, 'poor': 0.7},
        {'good': 0.9, 'fair': 0.7, 'poor': 0.5},
        {'good': 1.3, 'fair': 0.9, 'poor': 0.6}
    ]

    # Define inverse relationship for each financial ratio
    inverse_relationships = [False, True, False, True, False]

    # Iterate over each company and calculate credit scores for each financial ratio
    for company, ratios in company_ratios.items():
        print(f"Company {company}:")
        print("---" * 24)
        for i, ratio in enumerate(ratios):
            credit_score, credit_rating, confidence_level, rating_scales, range_scores = calculate_credit_score(
                ratio, industry_thresholds[i], expert_thresholds[i], inverse_relationships[i]
            )
            recommendation = get_credit_recommendation(credit_score, credit_rating, confidence_level)

            print(f"  Financial Ratio {i+1}:")
            print(f"    Credit Score: {credit_score}")
            print(f"    Credit Rating: {credit_rating}")
            print(f"    Confidence Level: {confidence_level:.2%}")
            print(f"    Range Scores:")
            for rating, scores in range_scores.items():
                print(f"      {rating.capitalize()}: {scores['min']:.2f} - {scores['max']:.2f}")
            print(f"    Recommendation: {recommendation}")
        print()

Company A:
------------------------------------------------------------------------
  Financial Ratio 1:
    Credit Score: 133
    Credit Rating: good
    Confidence Level: 100.00%
    Range Scores:
      Good: 100.00 - 100.00
      Fair: 50.00 - 99.00
      Poor: 0.00 - 49.00
    Recommendation: The entity has a good credit rating with a confidence level of 100.00%. The credit score of 133 indicates a low risk level.
  Financial Ratio 2:
    Credit Score: 263
    Credit Rating: good
    Confidence Level: 100.00%
    Range Scores:
      Good: 100.00 - 100.00
      Fair: 50.00 - 99.00
      Poor: 0.00 - 49.00
    Recommendation: The entity has a good credit rating with a confidence level of 100.00%. The credit score of 263 indicates a low risk level.
  Financial Ratio 3:
    Credit Score: 150
    Credit Rating: good
    Confidence Level: 100.00%
    Range Scores:
      Good: 100.00 - 100.00
      Fair: 50.00 - 99.00
      Poor: 0.00 - 49.00
    Recommendation: The entity has a good cred

In [99]:
# ... (previous code remains the same)
    # Define financial ratios for each company
company_ratios = {
    'A': [0.2, 0.4, 1.5, 0.6, 1.0],
    'B': [0.9, 1.1, 1.3, 0.7, 0.8],
    'C': [1.0, 0.6, 0.1, 0.9, 0.2]
}

# Define industry and expert thresholds for each financial ratio
industry_thresholds = [
    {'good': 1.0, 'fair': 0.7, 'poor': 0.4},
    {'good': 0.6, 'fair': 0.4, 'poor': 0.2},
    {'good': 1.2, 'fair': 0.9, 'poor': 0.6},
    {'good': 0.8, 'fair': 0.6, 'poor': 0.4},
    {'good': 1.1, 'fair': 0.8, 'poor': 0.5}
]

expert_thresholds = [
    {'good': 1.2, 'fair': 0.8, 'poor': 0.5},
    {'good': 0.7, 'fair': 0.5, 'poor': 0.3},
    {'good': 1.4, 'fair': 1.0, 'poor': 0.7},
    {'good': 0.9, 'fair': 0.7, 'poor': 0.5},
    {'good': 1.3, 'fair': 0.9, 'poor': 0.6}
]

# Define inverse relationship for each financial ratio
inverse_relationships = [False, True, False, True, False]

def calculate_overall_credit_score(credit_scores):
    """
    Calculate the overall credit score and rating based on individual financial ratio credit scores.

    Args:
        credit_scores (list): List of credit scores for each financial ratio.

    Returns:
        tuple: A tuple containing the overall credit score (float) and overall credit rating (str).
    """
    
    max_score = 100
    min_score = 0
    
    overall_credit_score = sum(credit_scores) / len(credit_scores)
    
    # Rescale the overall credit score to have a maximum rating of 100
    rescaled_credit_score = (overall_credit_score - min_score) / (max_score - min_score) * 100
    
    if overall_credit_score >= 80:
        overall_credit_rating = "Good"
    elif overall_credit_score >= 60:
        overall_credit_rating = "Fair"
    else:
        overall_credit_rating = "Poor"
    
    return overall_credit_score, overall_credit_rating

# Example usage
if __name__ == "__main__":
    
    # Iterate over each company and calculate credit scores for each financial ratio
    for company, ratios in company_ratios.items():
        print(f"Company {company}:")
        credit_scores = []
        
        for i, ratio in enumerate(ratios):
            credit_score, credit_rating, confidence_level, rating_scales, range_scores = calculate_credit_score(
                ratio, industry_thresholds[i], expert_thresholds[i], inverse_relationships[i]
            )
            recommendation = get_credit_recommendation(credit_score, credit_rating, confidence_level)
            
            credit_scores.append(credit_score)
            
            print(f"  Financial Ratio {i+1}:")
            print(f"    Credit Score: {credit_score}")
            print(f"    Credit Rating: {credit_rating}")
            print(f"    Confidence Level: {confidence_level:.2%}")
            print(f"    Range Scores:")
            for rating, scores in range_scores.items():
                print(f"      {rating.capitalize()}: {scores['min']:.2f} - {scores['max']:.2f}")
            print(f"    Recommendation: {recommendation}")
        
        overall_credit_score, overall_credit_rating = calculate_overall_credit_score(credit_scores)
        print(f"  Overall Credit Score: {overall_credit_score:.2f}")
        print(f"  Overall Credit Rating: {overall_credit_rating}")
        print()

Company A:
  Financial Ratio 1:
    Credit Score: 33
    Credit Rating: poor
    Confidence Level: 58.33%
    Range Scores:
      Good: 100.00 - 100.00
      Fair: 50.00 - 99.00
      Poor: 0.00 - 49.00
    Recommendation: The entity has a poor credit rating with a confidence level of 58.33%. The credit score of 33 indicates a high risk level.
  Financial Ratio 2:
    Credit Score: 575
    Credit Rating: good
    Confidence Level: 100.00%
    Range Scores:
      Good: 100.00 - 100.00
      Fair: 50.00 - 99.00
      Poor: 0.00 - 49.00
    Recommendation: The entity has a good credit rating with a confidence level of 100.00%. The credit score of 575 indicates a low risk level.
  Financial Ratio 3:
    Credit Score: 150
    Credit Rating: good
    Confidence Level: 100.00%
    Range Scores:
      Good: 100.00 - 100.00
      Fair: 50.00 - 99.00
      Poor: 0.00 - 49.00
    Recommendation: The entity has a good credit rating with a confidence level of 100.00%. The credit score of 150 indica

In [95]:
# ... (previous code remains the same)

def calculate_overall_credit_score(credit_scores):
    """
    Calculate the overall credit score and rating based on individual financial ratio credit scores.

    Args:
        credit_scores (list): List of credit scores for each financial ratio.

    Returns:
        tuple: A tuple containing the overall credit score (float) and overall credit rating (str).
    """
    max_score = 100
    min_score = 0
    
    overall_credit_score = sum(credit_scores) / len(credit_scores)
    
    # Rescale the overall credit score to have a maximum rating of 100
    rescaled_credit_score = (overall_credit_score - min_score) / (max_score - min_score) * 100
    
    if rescaled_credit_score >= 80:
        overall_credit_rating = "Good"
    elif rescaled_credit_score >= 60:
        overall_credit_rating = "Fair"
    else:
        overall_credit_rating = "Poor"
    
    return rescaled_credit_score, overall_credit_rating

# Example usage
if __name__ == "__main__":
    # ... (company ratios, industry thresholds, expert thresholds, and inverse relationships remain the same)
    
    # Iterate over each company and calculate credit scores for each financial ratio
    for company, ratios in company_ratios.items():
        print(f"Company {company}:")
        credit_scores = []
        
        for i, ratio in enumerate(ratios):
            credit_score, credit_rating, confidence_level, rating_scales, range_scores = calculate_credit_score(
                ratio, industry_thresholds[i], expert_thresholds[i], inverse_relationships[i]
            )
            recommendation = get_credit_recommendation(credit_score, credit_rating, confidence_level)
            
            credit_scores.append(credit_score)
            
            print(f"  Financial Ratio {i+1}:")
            print(f"    Credit Score: {credit_score}")
            print(f"    Credit Rating: {credit_rating}")
            print(f"    Confidence Level: {confidence_level:.2%}")
            print(f"    Range Scores:")
            for rating, scores in range_scores.items():
                print(f"      {rating.capitalize()}: {scores['min']:.2f} - {scores['max']:.2f}")
            print(f"    Recommendation: {recommendation}")
        
        overall_credit_score, overall_credit_rating = calculate_overall_credit_score(credit_scores)
        print(f"  Overall Credit Score: {overall_credit_score:.2f}")
        print(f"  Overall Credit Rating: {overall_credit_rating}")
        print()

Company A:
  Financial Ratio 1:
    Credit Score: 133
    Credit Rating: good
    Confidence Level: 100.00%
    Range Scores:
      Good: 100.00 - 100.00
      Fair: 50.00 - 99.00
      Poor: 0.00 - 49.00
    Recommendation: The entity has a good credit rating with a confidence level of 100.00%. The credit score of 133 indicates a low risk level.
  Financial Ratio 2:
    Credit Score: 263
    Credit Rating: good
    Confidence Level: 100.00%
    Range Scores:
      Good: 100.00 - 100.00
      Fair: 50.00 - 99.00
      Poor: 0.00 - 49.00
    Recommendation: The entity has a good credit rating with a confidence level of 100.00%. The credit score of 263 indicates a low risk level.
  Financial Ratio 3:
    Credit Score: 150
    Credit Rating: good
    Confidence Level: 100.00%
    Range Scores:
      Good: 100.00 - 100.00
      Fair: 50.00 - 99.00
      Poor: 0.00 - 49.00
    Recommendation: The entity has a good credit rating with a confidence level of 100.00%. The credit score of 150 ind

In [68]:
from scipy.spatial import distance

def calculate_credit_score(financial_ratio, industry_thresholds, expert_thresholds, inverse_relationship=False):
    """
    Calculate the credit score, rating, confidence level, and range scores based on a single financial ratio and thresholds.

    Args:
        financial_ratio (float): Financial ratio value.
        industry_thresholds (dict): Dictionary of industry thresholds for each rating.
        expert_thresholds (dict): Dictionary of expert thresholds for each rating.
        inverse_relationship (bool): Indicates if the financial ratio has an inverse relationship (lower is better).

    Returns:
        tuple: A tuple containing the credit score (float), credit rating (str),
               confidence level (float), rating scales (dict), and range scores (dict).
    """
    if inverse_relationship:
        financial_ratio = 1 / financial_ratio

    distances = {
        rating: distance.euclidean(
            [financial_ratio],
            [(industry_thresholds[rating] + expert_thresholds[rating]) / 2]
        )
        for rating in ['good', 'fair', 'poor']
    }

    credit_rating = min(distances, key=distances.get)

    max_score = 100
    min_score = 0
    min_threshold = min(industry_thresholds.values())
    max_threshold = max(industry_thresholds.values())

    credit_score = (financial_ratio - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score
    credit_score = max(min(credit_score, max_score), min_score)  # Ensure credit score is within the range [0, 100]

    confidence_level = 1 - distances[credit_rating] / (max_threshold - min_threshold)

    range_scores = {
        'good': {
            'min': (industry_thresholds['good'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score,
            'max': max_score
        },
        'fair': {
            'min': (industry_thresholds['fair'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score,
            'max': (industry_thresholds['good'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score
        },
        'poor': {
            'min': min_score,
            'max': (industry_thresholds['fair'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score
        }
    }

    return credit_score, credit_rating, confidence_level, range_scores

In [71]:
financial_ratio = 1.2
industry_thresholds = {'good': 2.0, 'fair': 0.7, 'poor': 0.4}
expert_thresholds = {'good': 2.0, 'fair': 0.8, 'poor': 0.5}

In [72]:
calculate_credit_score(financial_ratio, industry_thresholds, expert_thresholds)

(49.99999999999999,
 'fair',
 0.71875,
 {'good': {'min': 100.0, 'max': 100},
  'fair': {'min': 18.749999999999993, 'max': 100.0},
  'poor': {'min': 0, 'max': 18.749999999999993}})

In [100]:
33 / 39

0.8461538461538461